In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-83089c98-645f-800a-82c2-68207fb1e9b9)


In [12]:
!pip install tensorflow-gpu==2.1.0
!pip install -U git+https://github.com/qubvel/efficientnet

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 450kB 42.8MB/s 
     |████████████████████████████████| 3.8MB 50.0MB/s 
     |████████████████████████████████| 81kB 13.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goo

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-2qi8z3qv
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-2qi8z3qv
  Created wheel for efficientnet: filename=efficientnet-1.0.0-cp36-none-any.whl size=17686 sha256=592037c8a01cdafbb8c6453cbe3261a258f4378d865a3132e4c238bfc7b13eb1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ygmfdd0k/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet
  Found existing installation: efficientnet 1.0.0
    Uninstalling efficientnet-1.0.0:
      Successfully uninstalled efficientnet-1.0.0


In [3]:
!wget https://www.dropbox.com/s/ooyb72fftf4gy81/train_DETg9GD.zip
!wget https://www.dropbox.com/s/zzzabhbmr5mi0bo/test_Bh8pGW3.zip

--2020-01-21 10:48:42--  https://www.dropbox.com/s/ooyb72fftf4gy81/train_DETg9GD.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ooyb72fftf4gy81/train_DETg9GD.zip [following]
--2020-01-21 10:48:42--  https://www.dropbox.com/s/raw/ooyb72fftf4gy81/train_DETg9GD.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7cff1f0dd137f4f9afff0c2751.dl.dropboxusercontent.com/cd/0/inline/Awm7UgSibsG6bJY50WFxmYoBLTWC19GcHpg2oSOcKwaIS6NtWo71M5kzp3TpRDmBAIfS_Rcm5enGPueL3EnscehRqexxIiKU276BcrC0OynnVE4VF4a7qdPMYUSbigb3V5g/file# [following]
--2020-01-21 10:48:43--  https://uc7cff1f0dd137f4f9afff0c2751.dl.dropboxusercontent.com/cd/0/inline/Awm7UgSibsG6bJY50WFxmYoBLTWC19GcHpg2oSOcKwaIS6NtWo71M5kzp3TpRDmBAIfS_Rcm5enGPueL3EnscehRqexxIiKU276

In [0]:
!unzip -q "/content/train_DETg9GD.zip"
!unzip -q "/content/test_Bh8pGW3.zip"

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [13]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

ValueError: ignored

In [0]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

In [8]:
import pandas as pd

Train = pd.read_csv("/content/train.csv")
Train.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [10]:
train_image_paths = list(Train.ID)
train_image_labels = list(Train.Class)

print(train_image_paths[:10])
print(train_image_labels[:10])

['377.jpg', '17814.jpg', '21283.jpg', '16496.jpg', '4487.jpg', '6283.jpg', '23495.jpg', '7100.jpg', '6028.jpg', '22617.jpg']
['MIDDLE', 'YOUNG', 'MIDDLE', 'YOUNG', 'MIDDLE', 'MIDDLE', 'YOUNG', 'YOUNG', 'YOUNG', 'OLD']


In [11]:
from sklearn.model_selection import train_test_split 

Xtrain , Xtest , Ytrain , Ytest = train_test_split(train_image_paths , train_image_labels , test_size = 0.1 , random_state = 2)

print(Xtrain[:10])
print(Xtest[:10])
print(Ytrain[:10])
print(Ytest[:10])

['16143.jpg', '21347.jpg', '18741.jpg', '2009.jpg', '23735.jpg', '10328.jpg', '25596.jpg', '18886.jpg', '24000.jpg', '9626.jpg']
['20902.jpg', '14877.jpg', '6649.jpg', '6870.jpg', '11367.jpg', '19407.jpg', '11245.jpg', '4084.jpg', '16844.jpg', '10311.jpg']
['MIDDLE', 'YOUNG', 'MIDDLE', 'YOUNG', 'MIDDLE', 'MIDDLE', 'MIDDLE', 'MIDDLE', 'MIDDLE', 'MIDDLE']
['YOUNG', 'MIDDLE', 'MIDDLE', 'MIDDLE', 'MIDDLE', 'YOUNG', 'MIDDLE', 'MIDDLE', 'OLD', 'MIDDLE']


In [0]:
!mkdir data

In [0]:
def isTest(path : str):
    if path.split("/")[2] == "test":
        return True 
    else :
        return False

def isTrain(path : str):
    if path.split("/")[2] == "train":
        return True 
    else :
        return False

In [14]:
Test = pd.read_csv("/content/test.csv")
Test.head()

,ID
0,25321.jpg
1,989.jpg
2,19277.jpg
3,13093.jpg
4,5367.jpg


In [0]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import tempfile

class LearningRateFinder:
	def __init__(self, model, stopFactor=4, beta=0.98):
		self.model = model
		self.stopFactor = stopFactor
		self.beta = beta

		self.lrs = []
		self.losses = []

		self.lrMult = 1
		self.avgLoss = 0
		self.bestLoss = 1e9
		self.batchNum = 0
		self.weightsFile = None

	def reset(self):
		self.lrs = []
		self.losses = []
		self.lrMult = 1
		self.avgLoss = 0
		self.bestLoss = 1e9
		self.batchNum = 0
		self.weightsFile = None

	def is_data_iter(self, data):
		iterClasses = ["NumpyArrayIterator", "DirectoryIterator",
			 "DataFrameIterator", "Iterator", "Sequence"]

		return data.__class__.__name__ in iterClasses

	def on_batch_end(self, batch, logs):
		lr = K.get_value(self.model.optimizer.lr)
		self.lrs.append(lr)
  
		l = logs["loss"]
		self.batchNum += 1
		self.avgLoss = (self.beta * self.avgLoss) + ((1 - self.beta) * l)
		smooth = self.avgLoss / (1 - (self.beta ** self.batchNum))
		self.losses.append(smooth)

		stopLoss = self.stopFactor * self.bestLoss

		if self.batchNum > 1 and smooth > stopLoss:
			self.model.stop_training = True
			return

		if self.batchNum == 1 or smooth < self.bestLoss:
			self.bestLoss = smooth

		lr *= self.lrMult
		K.set_value(self.model.optimizer.lr, lr)

	def find(self, trainData, startLR, endLR, epochs=None,
		stepsPerEpoch=None, batchSize=32, sampleSize=2048,
		classWeight=None, verbose=1):

		self.reset()

		useGen = self.is_data_iter(trainData)

		if useGen and stepsPerEpoch is None:
			msg = "Using generator without supplying stepsPerEpoch"
			raise Exception(msg)

		elif not useGen:
			numSamples = len(trainData[0])
			stepsPerEpoch = np.ceil(numSamples / float(batchSize))

		if epochs is None:
			epochs = int(np.ceil(sampleSize / float(stepsPerEpoch)))

		numBatchUpdates = epochs * stepsPerEpoch

		self.lrMult = (endLR / startLR) ** (1.0 / numBatchUpdates)

		self.weightsFile = tempfile.mkstemp()[1]
		self.model.save_weights(self.weightsFile)

		origLR = K.get_value(self.model.optimizer.lr)
		K.set_value(self.model.optimizer.lr, startLR)
  
		callback = LambdaCallback(on_batch_end=lambda batch, logs:
			self.on_batch_end(batch, logs))

		if useGen:
			self.model.fit_generator(
				trainData,
				steps_per_epoch=stepsPerEpoch,
				epochs=epochs,
				class_weight=classWeight,
				verbose=verbose,
				callbacks=[callback])

		else:
			self.model.fit(
				trainData[0], trainData[1],
				batch_size=batchSize,
				epochs=epochs,
				class_weight=classWeight,
				callbacks=[callback],
				verbose=verbose)

		self.model.load_weights(self.weightsFile)
		K.set_value(self.model.optimizer.lr, origLR)

	def plot_loss(self, skipBegin=10, skipEnd=1, title=""):
		lrs = self.lrs[skipBegin:-skipEnd]
		losses = self.losses[skipBegin:-skipEnd]

		plt.plot(lrs, losses)
		plt.xscale("log")
		plt.xlabel("Learning Rate (Log Scale)")
		plt.ylabel("Loss")

		if title != "":
			plt.title(title)

In [0]:
from tensorflow.keras.callbacks import *
import tensorflow.keras.backend as K

class CyclicLR(Callback):
	def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
				 gamma=1., scale_fn=None, scale_mode='cycle'):
		super(CyclicLR, self).__init__()

		self.base_lr = base_lr
		self.max_lr = max_lr
		self.step_size = step_size
		self.mode = mode
		self.gamma = gamma
		if scale_fn == None:
			if self.mode == 'triangular':
				self.scale_fn = lambda x: 1.
				self.scale_mode = 'cycle'
			elif self.mode == 'triangular2':
				self.scale_fn = lambda x: 1 / (2. ** (x - 1))
				self.scale_mode = 'cycle'
			elif self.mode == 'exp_range':
				self.scale_fn = lambda x: gamma ** (x)
				self.scale_mode = 'iterations'
		else:
			self.scale_fn = scale_fn
			self.scale_mode = scale_mode
		self.clr_iterations = 0.
		self.trn_iterations = 0.
		self.history = {}

		self._reset()

	def _reset(self, new_base_lr=None, new_max_lr=None,
			   new_step_size=None):
		"""Resets cycle iterations.
		Optional boundary/step size adjustment.
		"""
		if new_base_lr != None:
			self.base_lr = new_base_lr
		if new_max_lr != None:
			self.max_lr = new_max_lr
		if new_step_size != None:
			self.step_size = new_step_size
		self.clr_iterations = 0.

	def clr(self):
		cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
		x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
		if self.scale_mode == 'cycle':
			return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
		else:
			return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
				self.clr_iterations)

	def on_train_begin(self, logs={}):
		logs = logs or {}

		if self.clr_iterations == 0:
			K.set_value(self.model.optimizer.lr, self.base_lr)
		else:
			K.set_value(self.model.optimizer.lr, self.clr())

	def on_batch_end(self, epoch, logs=None):

		logs = logs or {}
		self.trn_iterations += 1
		self.clr_iterations += 1

		self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
		self.history.setdefault('iterations', []).append(self.trn_iterations)

		for k, v in logs.items():
			self.history.setdefault(k, []).append(v)

		K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
import os
import shutil

def AllocateThings(Paths , Labels , Train = True):
    rem = ""
    if not os.path.isdir("data"):
         os.mkdir("data")
    if Train :
        os.mkdir("data/train")
        rem = "data/train/"
    else :
        os.mkdir("data/test")
        rem = "data/test/"
    unique_labels = list(set(Labels))
    for i in unique_labels :
        os.mkdir(rem + str(i))
    for i , j in zip(Paths , Labels):
        shutil.move("Train/" + i , rem + str(j) + "/")

In [0]:
AllocateThings(Xtrain , Ytrain , Train = True)
AllocateThings(Xtest , Ytest , Train = False)

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.25,
        zoom_range=0.2, 
        horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical')

Found 17915 images belonging to 3 classes.
Found 1991 images belonging to 3 classes.


In [22]:
import efficientnet.tfkeras as efn 
from tensorflow.keras.layers import Dense , Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

def get_model():
    #weights='imagenet',
    #base_model = ResNet50V2(include_top=False,input_shape=(64, 64, 3), pooling=None)
    base_model =  efn.EfficientNetB0(include_top=False, pooling='avg', input_shape=(64, 64, 3))
    x = base_model.output
    print('Basemodel dtype: %s' % x.dtype.name)
    x = Dense(128 , activation='elu')(x)
    x = Dropout(0.3)(x)
    y_pred = Dense(3, activation='softmax' , dtype = 'float32')(x)
    print('Outputs dtype: %s' % y_pred.dtype.name)
    return Model(inputs=base_model.input, outputs=y_pred)

model = get_model()

Basemodel dtype: float16
Outputs dtype: float32


In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint , LearningRateScheduler , ReduceLROnPlateau

def getCallBacks():
	checkpoint = ModelCheckpoint(filepath="model.h5",
	                             monitor='val_accuracy',
	                             verbose=1,
	                             save_best_only=True,save_weights_only = True)
 
	callbacks = [checkpoint]

	return callbacks

In [0]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy',metrics = ['accuracy'])

In [0]:
# modes can be "triangular" , "triangular2" , "exp_range"

clr = CyclicLR(
	mode = "triangular",
	base_lr = 1e-4,
	max_lr = 1e-2,
	step_size = 3 * len(train_generator))

In [26]:
train_generator.class_indices

{'MIDDLE': 0, 'OLD': 1, 'YOUNG': 2}

In [27]:
trainY = np.zeros((len(Xtrain) , len(train_generator.class_indices)))

for i , e in enumerate(Ytrain):
    trainY[i , train_generator.class_indices[e]] = 1

print(trainY.shape)

(17915, 3)


In [28]:
classTotals = trainY.sum(axis=0)
classWeight = classTotals.max() / classTotals

print(classWeight)

[1.         4.49976863 1.61260365]


**Phase 1**

In [0]:
callbacks = getCallBacks()
callbacks.append(clr)

In [30]:
model.fit(train_generator,steps_per_epoch=len(train_generator), epochs = 18 , verbose = 1 ,validation_data=validation_generator,validation_steps=len(validation_generator) , callbacks = callbacks , class_weight = classWeight)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 280 steps, validate for 32 steps
Epoch 1/18
279/280 [============================>.] - ETA: 0s - loss: 1.0540 - accuracy: 0.5113
Epoch 00001: val_accuracy improved from -inf to 0.58413, saving model to model.h5
280/280 [==============================] - 111s 398ms/step - loss: 1.0538 - accuracy: 0.5116 - val_loss: 0.8960 - val_accuracy: 0.5841
Epoch 2/18
279/280 [============================>.] - ETA: 0s - loss: 0.7954 - accuracy: 0.6498
Epoch 00002: val_accuracy improved from 0.58413 to 0.69161, saving model to model.h5
280/280 [==============================] - 94s 336ms/step - loss: 0.7949 - accuracy: 0.6501 - val_loss: 0.7174 - val_accuracy: 0.6916
Epoch 3/18
279/280 [============================>.] - ETA: 0s - loss: 0.6970 - accuracy: 0.6989
Epoch 00003: val_accuracy improved from 0.69161 to 0.72024, saving model to model.h5
280/280 [==============================] - 93s 334ms/step - loss: 0.6965 - accuracy: 0.6992 - val_

In [0]:
model.load_weights("model.h5")

**Phase 2**